<a href="https://colab.research.google.com/github/sarisppp/PJ63_CE18/blob/master/test_paramitor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

from sklearn.preprocessing import MinMaxScaler

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
used_features = ['Timestamp','SET50.Close','EMAV(34)','RSI(14)','EMAV(RSI,13)','MACD(13,5,5)','Signal']
data = pd.read_excel("/content/drive/My Drive/Set50_20190314_20200820_1minute (1).xlsx",usecols =used_features)
data.set_index("Timestamp",inplace=True)
data=data.dropna()

In [33]:
def signal(data,periods=16):
  data['output']=0
  for index,row in data.iterrows():
    if row['MACD(13,5,5)']>row['Signal'] and row['RSI(14)']>row['EMAV(RSI,13)'] and row['RSI(14)']<70:
      signalPre=1
    elif row['MACD(13,5,5)']<row['Signal'] and row['RSI(14)']<row['EMAV(RSI,13)'] and row['RSI(14)']>30:
      if row['SET50.Close']<row['EMAV(34)']:
        signalPre=-1
      else:
        signalPre=0
    else:
      signalPre=0
    data.at[index,'output']= signalPre
    
     
  return data

In [54]:
data_new=data
data_new=signal(data_new)
data_new=data_new.head(65)
data_new.info()

    

<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, 13/03/19 14:59:00 to 13/03/19 16:03:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SET50.Close   65 non-null     float64
 1   EMAV(34)      65 non-null     float64
 2   RSI(14)       65 non-null     float64
 3   MACD(13,5,5)  65 non-null     float64
 4   Signal        65 non-null     float64
 5   EMAV(RSI,13)  65 non-null     float64
 6   output        65 non-null     int64  
dtypes: float64(6), int64(1)
memory usage: 4.1+ KB


In [55]:
data_new['buy']=0
data_new['sell']=0
data_new['status']=0
data_new['close']=0

data_new=data_new.astype(float)
status=0
for index,row in data_new.iterrows():
  if status==0:
      if row['output']==1:
        status=1
        close=row['SET50.Close']
        buy=0
        sell=0
      elif row['output']==-1:
        status=-1
        close=row['SET50.Close']
        buy=0
        sell=0
      else:
        status=0
        buy=0
        sell=0
  else :
    if status==1:
      if row['output']==1:
        buy=0
        sell=0
      elif row['output']==-1:
        status=-1
        sell=row['SET50.Close']-close
        close=row['SET50.Close']
        buy=0
      else:
        buy=0
        sell=0
    elif status==-1:
       if row['output']==1:
        status=1
        buy=close-row['SET50.Close']
        close=row['SET50.Close']
        sell=0
       elif row['output']==-1:
        buy=0
        sell=0
       else:
        buy=0
        sell=0
    else:
      buy=0
      sell=0
  data_new.at[index,'buy']=buy
  data_new.at[index,'sell']=sell
  data_new.at[index,'status']=status
  data_new.at[index,'close']=close


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [56]:
data_new.info()
data_new

<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, 13/03/19 14:59:00 to 13/03/19 16:03:00
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SET50.Close   65 non-null     float64
 1   EMAV(34)      65 non-null     float64
 2   RSI(14)       65 non-null     float64
 3   MACD(13,5,5)  65 non-null     float64
 4   Signal        65 non-null     float64
 5   EMAV(RSI,13)  65 non-null     float64
 6   output        65 non-null     float64
 7   buy           65 non-null     float64
 8   sell          65 non-null     float64
 9   status        65 non-null     float64
 10  close         65 non-null     float64
dtypes: float64(11)
memory usage: 8.6+ KB


,SET50.Close,EMAV(34),RSI(14),"MACD(13,5,5)",Signal,"EMAV(RSI,13)",output,buy,sell,status,close
Timestamp,,,,,,,,,,,
13/03/19 14:59:00,1081.74,1081.6691,49.0514,0.0831,0.2186,53.734,0.0,0.00,0.00,0.0,1082.69
13/03/19 15:00:00,1082.10,1081.6937,52.0825,0.0583,0.1652,53.498,0.0,0.00,0.00,0.0,1082.69
13/03/19 15:01:00,1081.04,1081.6564,43.8165,-0.1605,0.0566,52.115,-1.0,0.00,0.00,-1.0,1081.04
13/03/19 15:02:00,1082.73,1081.7177,55.8480,0.0440,0.0524,52.648,0.0,0.00,0.00,-1.0,1081.04
13/03/19 15:03:00,1082.66,1081.7716,55.3196,0.1454,0.0834,53.030,1.0,-1.62,0.00,1.0,1082.66
...,...,...,...,...,...,...,...,...,...,...,...
13/03/19 15:59:00,1085.51,1085.2188,50.0572,0.0189,0.0794,55.876,0.0,0.00,0.00,1.0,1082.69
13/03/19 16:00:00,1085.87,1085.2560,52.4307,-0.0090,0.0500,55.384,0.0,0.00,0.00,1.0,1082.69
13/03/19 16:01:00,1085.93,1085.2945,52.8330,-0.0130,0.0290,55.020,0.0,0.00,0.00,1.0,1082.69
